In [1]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm

In [2]:
data_dir = Path('/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data')
train_dir = data_dir/'train'
test_dir = data_dir/'test'

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load the full training dataset
full_train_data = datasets.ImageFolder(train_dir, transform=transform)

In [8]:
train_size = int(0.8 * len(full_train_data))
val_size = len(full_train_data) - train_size

train_data, val_data = random_split(full_train_data, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2)

print(f"Train size: {len(train_data)}, Validation size: {len(val_data)}")
print(f"Classes: {len(full_train_data.classes)}")

Train size: 6515, Validation size: 1629
Classes: 196


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ResNet18 and modify the final layer
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_classes = len(full_train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, 
                       steps_per_epoch=len(train_loader), epochs=5)

# Training config
num_epochs = 5
best_val_acc = 0.0
train_correct = 0
train_total = 0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_acc = train_correct / train_total
    print(f"Training Loss: {avg_train_loss:.4f}, Accuracy: {train_acc * 100:.2f}%")

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_acc * 100:.2f}%")

    # Save best model (optional)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_resnet50.pth")
        print("✅ Best model saved.")


Epoch 1/5


100%|██████████| 204/204 [00:52<00:00,  3.92it/s]

Training Loss: 48.3981, Accuracy: 4.28%


Validation Loss: 131.7078, Accuracy: 3.87%
✅ Best model saved.

Epoch 2/5


100%|██████████| 204/204 [00:37<00:00,  5.50it/s]

...

Training Loss: 18.8885, Accuracy: 23.28%


Validation Loss: 30.1025, Accuracy: 31.12%
✅ Best model saved.

Epoch 5/5


100%|██████████| 204/204 [00:37<00:00,  5.47it/s]

Training Loss: 4.3801, Accuracy: 33.73%


Validation Loss: 23.2744, Accuracy: 35.17%
✅ Best model saved.
